In [1]:
import numpy as np
import nrrd
import pydicom
from pydicom import dcmread
import os
import numpy
from os import listdir
from os.path import isfile, join
import pandas as pd
from ipywidgets import interact
import matplotlib.pyplot as plt
from PIL import Image as im
from matplotlib import pyplot, cm
#Ideas from: https://pynrrd.readthedocs.io/en/latest/examples.html

In [2]:
target_folder = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Duke-Breast-Cancer-MRI\Breast_MRI_190\01-01-1990-NA-MRI BREAST BILATERAL WWO-57539\600.000000-ax 3d dyn-08498"
mypath = target_folder
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
filepaths = [target_folder + "/" + s for s in onlyfiles]

mypath = target_folder
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
testdict = {}
sorteddict = {}
for x in onlyfiles: # for loop to make a dictionary with all the filenames
    tempfile = mypath + "/" + str(x)
    testds = dcmread(tempfile)
    position = testds.ImagePositionPatient[2]
    testdict.update({tempfile: position})

    #then put them in another dictionary ordered by z-coordinate by first retrieving the key and value for the highest value and then using a while loop to each time add the highest value to the sorted dictionary and then removing it from the first 
highestkey = max(testdict, key=testdict.get)
highestvalue = testdict[highestkey]       
while len(testdict) > 0:
    highestkey = max(testdict, key=testdict.get)
    highestvalue = testdict[highestkey]
    sorteddict.update({highestkey: highestvalue})
    testdict.pop(highestkey)

filetitles = list(sorteddict.keys())
RefDs = pydicom.read_file(filetitles[0])
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(filetitles))
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [3]:
def Make_mask(patientnr):
    df = pd.read_excel (r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Annotation_Boxes.xlsx')
    test = df.iloc[patientnr]
    maskstats = pd.Series(test).values
    ArrayMask = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

    for x in range (maskstats[1],maskstats[2]):
        for y in range (maskstats[3],maskstats[4]):
            for z in range (maskstats[5],maskstats[6]+1):
                ArrayMask[x,y,z] = 1

In [4]:
ArrayMask = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)
for patientnr in range (0,922):
    Make_mask(patientnr)
    target_location = r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset/Box_masks/'
    target_file = target_location + str(patientnr + 1) + ".nrrd"
    data = ArrayMask
    nrrd.write(target_file, data)